# Equities Technical Screening

Here, we'll do some technical screening and explore ways to combine various technical indicators in our equity universe

In [1]:
from ib_insync import *
import pandas as pd
import time
import nasdaqdatalink
import sys
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
from src.utils.technical_indicators import *

import os
from dotenv import load_dotenv
NASDAQ_KEY = os.getenv("NASDAQ_KEY")

nasdaqdatalink.ApiConfig.api_key = NASDAQ_KEY

In [2]:
tickers_list = [
    "RKDA", "LND", "CALM", "UAVS", "BA", "AIRI", "MFIN", "AVTA", "EEFT", "JSDA",
    "PRMW", "ZVIA", "ALTO", "ADES", "MARK", "FENG", "IDEX", "AREN", "CCOI", "IMMR",
    "CLSK", "OMQS", "INTZ", "ORN", "WLGS", "UHG", "SKY", "RUN", "PGTI", "BSET",
    "CTHR", "PRPL", "SWBI", "RDW", "SPB", "BYRN", "NCNC", "GDEN", "TOON", "GAIA",
    "BGLC", "LFMD", "LSTA", "ESGR", "KFS", "GLRE", "IVAC", "EKSO", "SMTI", "MDGS",
    "ASXC", "REPX", "NOG", "YPF", "ENZN", "PRPH", "BLTE", "CTGO", "NEWP", "ORLA",
    "STRS", "INTG", "MLP", "MICS", "GNSS", "RTC", "RAIL", "MBUU", "CCK", "OI",
    "XXII", "VGR", "PSHG", "USDP", "ELLO", "KEN", "NGL", "VRTV", "VOXX"
]

"""
For our universe of stocks to screen, these were gathered from a scoring process with fundamental
indicators. We want to test various technical indicators individually and in combination on this
universe to see what we find. 

First, we'll load a dictionary of dataframes with price data.
"""

universe_prices = {}

for ticker in tickers_list:

    prices = nasdaqdatalink.get_table('SHARADAR/SEP', qopts={"columns":['date', 'close']}, date={'gte':'2023-01-01'}, ticker=ticker)
    prices_df = pd.DataFrame(prices[::-1])
    prices_df["log_returns"] = np.log(prices_df.close / prices_df.close.shift(1))
    universe_prices[ticker] = prices_df



KeyboardInterrupt: 

# RSI 

Let's start with RSI. Here, we'll explore different ways to interpret just the RSI value alone.


In [ ]:
# Lets screen for tickers that have been oversold so RSI should be in the 0-30 range

rsi_below_30 = []

for ticker in tickers_list:
    # We'll use the utitlity function here to add the RSI column to our prices dataframe
    rsi_df = relative_strength_index(universe_prices[ticker])
    last_row_rsi = rsi_df.iloc[-1]
    if (last_row_rsi['rsi'] < 30 and last_row_rsi['rsi'] > 0):
        rsi_below_30.append(ticker)

print('RSI Below 30 (Oversold): ', rsi_below_30)


RSI Below 30 (Oversold):  ['MARK', 'FENG', 'IDEX', 'CTHR', 'SWBI', 'NCNC', 'SMTI', 'YPF', 'PRPH', 'INTG', 'MICS', 'RAIL', 'MBUU', 'USDP']


# MACD

Lets look for various ways to screen for the MACD

In [ ]:
# Lets screen for tickers where the MACD is above the signal line

macd_above_signal = []

for ticker in tickers_list:
    macd_df = macd(universe_prices[ticker])
    last_row_macd = macd_df.iloc[-1]
    if (last_row_macd['macd'] > last_row_macd['signal']):
        macd_above_signal.append(ticker)

print("MACD Line greater than signal line: ", macd_above_signal)

MACD Line greater than signal line:  ['RKDA', 'LND', 'CALM', 'UAVS', 'AIRI', 'AVTA', 'EEFT', 'JSDA', 'ZVIA', 'MARK', 'FENG', 'AREN', 'OMQS', 'INTZ', 'ORN', 'WLGS', 'UHG', 'SKY', 'RUN', 'PRPL', 'SPB', 'BYRN', 'GDEN', 'TOON', 'GAIA', 'LFMD', 'EKSO', 'MDGS', 'ASXC', 'REPX', 'NOG', 'ENZN', 'CTGO', 'GNSS', 'RTC', 'CCK', 'OI', 'XXII', 'VGR', 'PSHG', 'VOXX']
